<img width="8%" alt="Plotly.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Plotly.png" style="border-radius: 15%">

# Plotly - Follow meetings and tasks

**Tags:** #plotly #html #csv #image #operations #analytics #meetings #tasks #metric

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook creates a barchart graph tracking your daily meetings and tasks over the past two weeks.

## Input

### Import libraries

In [ ]:
import plotly.graph_objects as go
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date, datetime, timedelta
import naas_data_product
import random

### Setup variables
**Inputs**
- `entity_dir`: Entity directory.
- `entity_name`: Entity name.
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Input file.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `title`: Graph title.

**Outputs**
- `output_dir`: This variable is used for storing the path to the directory where the output files will be saved.

In [ ]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
input_dir = os.path.join(entity_dir, "operations-engine", date.today().isoformat())
input_file = "meetings"
input_file2 = "tasks"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name = "MEETINGS"
sheet_name2 = "TASKS"
title = "Ops"

# Outputs
output_dir = os.path.join(entity_dir, "operations-engine", date.today().isoformat())
os.makedirs(output_dir, exist_ok=True)
output_name = "ops_trend"

## Model

### Set outputs

In [ ]:
html_output = os.path.join(output_dir, f"{output_name}.html")
image_output = os.path.join(output_dir, f"{output_name}.png")

### Get input data

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
PERIOD = "%Y-%m-%d"
PERIOD_TEXT = "This day"

data1 = {
    "ENTITY": ["Abi"] * 10,
    "SCENARIO": ["W03-2024"] * 5 +  ["W04-2024"] * 5,
    "LABEL": [
        "2024-02-12",
        "2024-02-13",
        "2024-02-14",
        "2024-02-15",
        "2024-02-16",
        "2024-02-19",
        "2024-02-20",
        "2024-02-21",
        "2024-02-22",
        "2024-02-23",
    ],
    "GROUP": ["Meetings"] * 10,
    "VALUE": [random.randint(0, 5) for i in range(0, 10)],
}
df1 = pd.DataFrame(data1)

data2 = {
    "ENTITY": ["Abi"] * 10,
    "SCENARIO": ["W03-2024"] * 5 +  ["W04-2024"] * 5,
    "LABEL": [
        "2024-02-12",
        "2024-02-13",
        "2024-02-14",
        "2024-02-15",
        "2024-02-16",
        "2024-02-19",
        "2024-02-20",
        "2024-02-21",
        "2024-02-22",
        "2024-02-23",
    ],
    "GROUP": ["Tasks"] * 10,
    "VALUE": [random.randint(0, 10) for i in range(0, 10)],
}
df2 = pd.DataFrame(data2)
df_trend = pd.concat([df1, df2])
df_trend["LABEL_D"] = pd.to_datetime(df_trend["LABEL"], format=PERIOD).dt.strftime("%a %d %b")
df_trend["COLOR"] = df_trend["GROUP"].map({"Meetings": "grey", "Tasks": "lightgrey"})
df_trend

### Create title and logo

In [ ]:
def get_kpis(df):
    # Groupby weeks
    if len(df) > 0 and "SCENARIO" in df.columns:
        df = df.groupby(["SCENARIO"], as_index=False).agg({"VALUE": "sum"})

    # Get total and total n-1
    if len(df) == 0:
        total = 0
        total_n1 = 0
    elif len(df) == 1:
        if df.loc[0, "SCENARIO"] == TW:
            total = df.loc[0, "VALUE"]
            total_n1 = 0
        else:
            total = 0
            total_n1 = df.loc[0, "VALUE"]
    else:
        total = df.loc[df.index[-1], "VALUE"]
        total_n1 = df.loc[df.index[-2], "VALUE"]

    # Calc variation in value and pourcentage 
    varv = total - total_n1
    if total == 0:
        varp = -1
    elif total_n1 == 0:
        varp = 1
    else:
        varp = varv / total_n1

    # Create value to displayed
    total_d = "{:,.0f}".format(total).replace(",", " ")
    varv_d = "{:,.0f}".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
        
    # Logo
    logo = None
    if varv > 0:
        logo = arrow_up
    elif varv > -0.2:
        logo = arrow_right
    else:
        logo = arrow_down
    
    return total_d, varv_d, varp_d, logo
        
total_d, varv_d, varp_d, logo = get_kpis(df1)
total_d2, varv_d2, varp_d2, logo2 = get_kpis(df2)
title_full = f"<b><span style='font-size: 20px;'>Meetings: {total_d} | {varv_d} ({varp_d})</span></b><br><span style='font-size: 18px;'>Tasks: {total_d2} | {varv_d2} ({varp_d2})</span>"
print("Title:", title_full)

### Create trend dataframe

In [ ]:
def create_barchart(df, label, groups, value, title):
    fig = go.Figure()

    list_groups = df[groups].unique()
    for group in list_groups:
        tmp_df = df[df[groups] == group]
        fig.add_trace(go.Bar(x=tmp_df[label], y=tmp_df[value], name=group, marker=dict(color=tmp_df["COLOR"]),))
        
    # Add logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.05,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom",
        )
    )
    # Add legend
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )
    
    # Update layout
    fig.update_layout(
        title=title_full,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        xaxis_tickfont_size=14,
        legend=dict(bgcolor="white", bordercolor="white"),
        barmode="group",
        bargap=0.1,  # gap between bars of adjacent location coordinates.
        bargroupgap=0.1,  # gap between bars of the same location coordinate.
    )
    config = {"displayModeBar": False}
    fig.show(config=config)
    return fig

fig = create_barchart(df_trend, label="LABEL_D", groups="GROUP", value="VALUE", title=title)

## Output

### Save and share your csv file

In [ ]:
pdump(output_dir, df_trend, output_name)

### Save and share your graph in HTML


In [ ]:
# Save your graph in HTML
fig.write_html(html_output)

# Share output with naas
html_link = naas.asset.add(html_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image


In [ ]:
# Save your graph in PNG
fig.write_image(image_output)

# Share output with naas
image_link = naas.asset.add(image_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)